In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, confusion_matrix

In [2]:
data=pd.read_csv('balanced_dataset_50_50.csv')

In [3]:
data.head()

,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,TRANSFER,129526.58,C1786736565,0.00,0.00,C1189777296,2103690.88,2233217.46,0.0
1,CASH_IN,304210.97,C2138211913,6638620.67,6942831.64,C1766776273,513871.60,209660.63,0.0
2,CASH_OUT,57905.63,C393598530,188.00,0.00,C769319919,611506.02,669411.65,0.0
3,CASH_IN,83102.70,C807731926,2274365.81,2357468.51,C322340400,122204.31,39101.61,0.0
4,PAYMENT,14341.19,C603392505,459181.00,444839.81,M1570710058,0.00,0.00,0.0


In [4]:
data.describe()

,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
count,5.070000e+03,5.070000e+03,5.070000e+03,5.070000e+03,5.070000e+03,5070.000000
mean,7.305437e+05,1.087883e+06,4.523101e+05,7.786253e+05,1.165501e+06,0.500000
std,1.605670e+06,2.672994e+06,2.136752e+06,2.252226e+06,2.612199e+06,0.500049
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
25%,3.361776e+04,1.011947e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
50%,1.677155e+05,1.133390e+05,0.000000e+00,0.000000e+00,1.438789e+05,0.500000
75%,5.223438e+05,7.980074e+05,0.000000e+00,5.533556e+05,1.152234e+06,1.000000
max,1.000000e+07,3.767004e+07,3.636535e+07,3.296166e+07,3.464570e+07,1.000000


In [5]:
data = data.drop(['nameOrig','nameDest'], axis=1)

In [6]:
encoder = OneHotEncoder(drop='first', sparse_output=False)

encoded_type = encoder.fit_transform(data[['type']])

In [7]:
encoded_type_df = pd.DataFrame(encoded_type,
                               columns=encoder.get_feature_names_out(['type']),
                               index=data.index)

# Dropping the original 'type' column and adding the encoded columns
data_encoded = pd.concat([data.drop('type', axis=1), encoded_type_df], axis=1)

# Display the first few rows to verify the result
print(data_encoded.head())

      amount  oldbalanceOrg  newbalanceOrig  oldbalanceDest  newbalanceDest  \
0  129526.58           0.00            0.00      2103690.88      2233217.46   
1  304210.97     6638620.67      6942831.64       513871.60       209660.63   
2   57905.63         188.00            0.00       611506.02       669411.65   
3   83102.70     2274365.81      2357468.51       122204.31        39101.61   
4   14341.19      459181.00       444839.81            0.00            0.00   

   isFraud  type_CASH_OUT  type_DEBIT  type_PAYMENT  type_TRANSFER  
0      0.0            0.0         0.0           0.0            1.0  
1      0.0            0.0         0.0           0.0            0.0  
2      0.0            1.0         0.0           0.0            0.0  
3      0.0            0.0         0.0           0.0            0.0  
4      0.0            0.0         0.0           1.0            0.0  


In [8]:
X = data_encoded.drop('isFraud', axis=1)  # Features (all columns except the target)
y = data_encoded['isFraud']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scaling the data (SVM and KNN need scaled data)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Decision Tree

In [9]:
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)

# Predictions
y_pred_dt = dt_model.predict(X_test)

# Evaluation
print("Decision Tree Classification Report:\n", classification_report(y_test, y_pred_dt))
print("Decision Tree Accuracy: ", accuracy_score(y_test, y_pred_dt))

Decision Tree Classification Report:
               precision    recall  f1-score   support

         0.0       0.99      0.98      0.99       774
         1.0       0.98      0.99      0.99       747

    accuracy                           0.99      1521
   macro avg       0.99      0.99      0.99      1521
weighted avg       0.99      0.99      0.99      1521

Decision Tree Accuracy:  0.9861932938856016


In [11]:
dt_roc_auc = roc_auc_score(y_test, dt_model.predict_proba(X_test)[:, 1])
print(f"Decision Tree ROC-AUC: {dt_roc_auc}")
conf_matrix = confusion_matrix(y_test, y_pred_dt)
print("Confusion Matrix:\n", conf_matrix)

Decision Tree ROC-AUC: 0.9863173624731485
Confusion Matrix:
 [[758  16]
 [  5 742]]


Logistic Regression

In [12]:
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train_scaled, y_train)

# Predictions
y_pred_lr = lr_model.predict(X_test_scaled)

# Evaluation
print("Logistic Regression Classification Report:\n", classification_report(y_test, y_pred_lr))
print("Logistic Regression Accuracy: ", accuracy_score(y_test, y_pred_lr))

Logistic Regression Classification Report:
               precision    recall  f1-score   support

         0.0       0.92      0.92      0.92       774
         1.0       0.92      0.92      0.92       747

    accuracy                           0.92      1521
   macro avg       0.92      0.92      0.92      1521
weighted avg       0.92      0.92      0.92      1521

Logistic Regression Accuracy:  0.9184746877054569


In [13]:
lr_roc_auc = roc_auc_score(y_test, lr_model.predict_proba(X_test_scaled)[:, 1])
print(f"Logistic Regression ROC-AUC: {lr_roc_auc}")
conf_matrix = confusion_matrix(y_test, y_pred_lr)
print("Confusion Matrix:\n", conf_matrix)

Logistic Regression ROC-AUC: 0.9757894627606031
Confusion Matrix:
 [[711  63]
 [ 61 686]]


Support Vector Machine

In [14]:
svm_model = SVC(kernel='linear', probability=True)  # You can change the kernel type
svm_model.fit(X_train_scaled, y_train)

# Predictions
y_pred_svm = svm_model.predict(X_test_scaled)

# Evaluation
print("SVM Classification Report:\n", classification_report(y_test, y_pred_svm))
print("SVM Accuracy: ", accuracy_score(y_test, y_pred_svm))

SVM Classification Report:
               precision    recall  f1-score   support

         0.0       0.86      0.95      0.90       774
         1.0       0.94      0.84      0.89       747

    accuracy                           0.90      1521
   macro avg       0.90      0.90      0.90      1521
weighted avg       0.90      0.90      0.90      1521

SVM Accuracy:  0.8961209730440499


In [15]:
svm_roc_auc = roc_auc_score(y_test, svm_model.predict_proba(X_test_scaled)[:, 1])
print(f"SVM ROC-AUC: {svm_roc_auc}")
conf_matrix = confusion_matrix(y_test, y_pred_svm)
print("Confusion Matrix:\n", conf_matrix)

SVM ROC-AUC: 0.9786674691876895
Confusion Matrix:
 [[736  38]
 [120 627]]


K-Nearest Neighbour

In [16]:
knn_model = KNeighborsClassifier(n_neighbors=5)  # You can tune 'n_neighbors'
knn_model.fit(X_train_scaled, y_train)

# Predictions
y_pred_knn = knn_model.predict(X_test_scaled)

# Evaluation
print("KNN Classification Report:\n", classification_report(y_test, y_pred_knn))
print("KNN Accuracy: ", accuracy_score(y_test, y_pred_knn))

KNN Classification Report:
               precision    recall  f1-score   support

         0.0       0.95      0.93      0.94       774
         1.0       0.93      0.95      0.94       747

    accuracy                           0.94      1521
   macro avg       0.94      0.94      0.94      1521
weighted avg       0.94      0.94      0.94      1521

KNN Accuracy:  0.9388560157790927


In [17]:
knn_roc_auc = roc_auc_score(y_test, knn_model.predict_proba(X_test_scaled)[:, 1])
print(f"KNN ROC-AUC: {knn_roc_auc}")
conf_matrix = confusion_matrix(y_test, y_pred_knn)
print("Confusion Matrix:\n", conf_matrix)

KNN ROC-AUC: 0.9820150541874647
Confusion Matrix:
 [[721  53]
 [ 40 707]]


In [18]:
import pickle
with open('Fraud_detection_dtree.pkl', 'wb') as f:
    pickle.dump(dt_model, f)

In [19]:
load_model=pickle.load(open('Fraud_detection_dtree.pkl', 'rb'))

In [20]:
model_score=load_model.score(X_test, y_test)
model_score

0.9861932938856016